In [10]:
import numpy as np 
import pandas as pd

In [11]:
shakespeareData = pd.read_csv('../data/raw/Shakespeare/Shakespeare_data.csv', index_col='Dataline') 
shakespeareData

,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
Dataline,,,,,
1,Henry IV,NaN,NaN,NaN,ACT I
2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
...,...,...,...,...,...
111392,A Winters Tale,38.0,5.3.180,LEONTES,"Lead us from hence, where we may leisurely"
111393,A Winters Tale,38.0,5.3.181,LEONTES,Each one demand an answer to his part
111394,A Winters Tale,38.0,5.3.182,LEONTES,Perform'd in this wide gap of time since first
